In [ ]:

import os
import random
from datetime import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [ ]:

def download_data(ticker: str, start: str, end: str) -> pd.DataFrame:
    df = yf.download(ticker, start=start, end=end, progress=False)
    if df.empty:
        raise ValueError("No data downloaded. Check ticker or date range.")
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
    df.dropna(inplace=True)
    return df


In [ ]:

def create_features_targets(df: pd.DataFrame, feature_cols: list, target_col: str, seq_len: int = 60):
    data = df[feature_cols].values
    targets = df[target_col].values
    X, y, dates = [], [], []
    for i in range(seq_len, len(df)):
        X.append(data[i - seq_len:i])
        y.append(targets[i])
        dates.append(df.index[i])
    return np.array(X), np.array(y), np.array(dates)


In [ ]:

def build_lstm_model(input_shape, units=50, dropout=0.2):
    model = Sequential([
        LSTM(units, return_sequences=True, input_shape=input_shape),
        Dropout(dropout),
        LSTM(units//2, return_sequences=False),
        Dropout(dropout),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model


In [ ]:

def evaluate(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return {"RMSE": rmse, "MAE": mae, "MAPE(%)": mape}

def plot_results(dates, true_prices, pred_prices, title="Stock Price Prediction"):
    plt.figure(figsize=(12,6))
    plt.plot(dates, true_prices, label="True Close")
    plt.plot(dates, pred_prices, label="Predicted Close")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:


ticker = "AAPL"  
start_date = "2015-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")
seq_len = 60
test_size = 0.2
epochs = 50
batch_size = 32
units = 64
dropout = 0.2


print(f"Downloading data for {ticker}...")
df = download_data(ticker, start_date, end_date)
features = ['Open','High','Low','Close','Volume']
target = 'Close'

scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[features])
scaled_df = pd.DataFrame(scaled, index=df.index, columns=features)

X, y, dates = create_features_targets(scaled_df, features, target, seq_len)

split_idx = int((1 - test_size) * len(X))
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]
dates_test = dates[split_idx:]

print(f"Train samples: {len(X_train)}, Test samples: {len(X_test)}")


model = build_lstm_model(input_shape=(seq_len, len(features)), units=units, dropout=dropout)
callbacks = [EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]

history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=epochs, batch_size=batch_size, callbacks=callbacks, verbose=2)


y_pred = model.predict(X_test).flatten()


def inverse_close(scaled_rows):
    return scaler.inverse_transform(scaled_rows)[:, features.index('Close')]

y_test_full = []
y_pred_full = []
for i in range(len(X_test)):
    last_row = X_test[i, -1, :].copy()
    true_row = last_row.copy()
    true_row[features.index('Close')] = y_test[i]
    pred_row = last_row.copy()
    pred_row[features.index('Close')] = y_pred[i]
    y_test_full.append(true_row)
    y_pred_full.append(pred_row)

y_test_inv = inverse_close(np.array(y_test_full))
y_pred_inv = inverse_close(np.array(y_pred_full))


scores = evaluate(y_test_inv, y_pred_inv)
print("Evaluation Metrics:")
for k, v in scores.items():
    print(f"{k}: {v:.4f}")


plot_results(dates_test, y_test_inv, y_pred_inv, title=f"{ticker} Stock Price Prediction (LSTM)")
